In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys

sys.path.append("../")

In [2]:
from __future__ import annotations

import os
import types

import pyproj
import xarray as xr
import pyresample.kd_tree

from src.mesoformer.typing import Any
from src.mesoformer.datasets.metadata import ERA5Enum, URMAEnum, CFDatasetEnum

_test_data = "../tests/data"

urma_store = os.path.join(_test_data, "urma.zarr")
urma_dvars = list(URMAEnum)

era5_store = os.path.join(_test_data, "era5.zarr")
era5_dvars = list(ERA5Enum)

print(era5_dvars, urma_dvars, sep="\n")

[ERA5Enum['geopotential'], ERA5Enum['specific_humidity'], ERA5Enum['temperature'], ERA5Enum['u_component_of_wind'], ERA5Enum['v_component_of_wind'], ERA5Enum['vertical_velocity']]
[URMAEnum['total_cloud_cover'], URMAEnum['ceiling'], URMAEnum['u_wind_component_10m'], URMAEnum['v_wind_component_10m'], URMAEnum['wind_speed_10m'], URMAEnum['wind_speed_gust'], URMAEnum['wind_direction_10m'], URMAEnum['temperature_2m'], URMAEnum['dewpoint_temperature_2m'], URMAEnum['specific_humidity_2m'], URMAEnum['surface_pressure'], URMAEnum['visibility'], URMAEnum['orography']]


In [3]:
_URMA_DATASET = xr.open_zarr(urma_store)
_URMA_DATASET

<xarray.Dataset>
Dimensions:                  (time: 2, y: 1597, x: 2345)
Coordinates:
  * time                     (time) datetime64[ns] 2019-01-02 2019-01-02T01:0...
Dimensions without coordinates: y, x
Data variables: (12/14)
    ceiling                  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    latitude                 (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    longitude                (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    ...                       ...
    u_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    visibility               (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [4]:
if not os.path.exists(era5_store):
    _google_store = "gs://weatherbench2/datasets/era5/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2"
    xr.open_zarr(_google_store)[era5_dvars].sel(time=_URMA_DATASET.time).to_zarr(era5_store, mode="w")

_ERA5_DATASET = xr.open_zarr(era5_store)
_ERA5_DATASET

<xarray.Dataset>
Dimensions:              (time: 2, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 1 2 3 5 7 10 ... 875 900 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Data variables:
    geopotential         (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [5]:
from src.mesoformer.datasets.core import IndependentABC, set_independent, is_independent


def get_urma() -> xr.Dataset:
    return set_independent(_URMA_DATASET.copy())


def get_era5() -> xr.Dataset:
    return set_independent(_ERA5_DATASET.copy())


ds = get_urma()


ds

<xarray.Dataset>
Dimensions:                  (T: 2, Z: 1, Y: 1597, X: 2345)
Coordinates:
    latitude                 (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.37
    longitude                (Y, X) float64 233.7 233.7 233.8 ... 300.9 301.0
  * time                     (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
    vertical                 (Z) <U7 'derived'
Dimensions without coordinates: T, Z, Y, X
Data variables:
    ceiling                  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    temperature_2m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    total_cloud_cover        (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    u_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    visibility               (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [6]:
ds = get_era5()
ds

<xarray.Dataset>
Dimensions:              (T: 2, Z: 37, Y: 721, X: 1440)
Coordinates:
    latitude             (Y, X) float32 90.0 90.0 90.0 ... -90.0 -90.0 -90.0
  * vertical             (Z) int64 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
    longitude            (Y, X) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time                 (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X
Data variables:
    geopotential         (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [7]:
from src.mesoformer.typing import StrPath, Sequence, DictStrAny, Mapping, Hashable, Iterable
from src.mesoformer.datasets.metadata import MetadataMixin
from src.mesoformer.datasets.metadata import T, Z, X, Y, LON, LAT, LVL, TIME


VariableLike = type[CFDatasetEnum] | CFDatasetEnum | Sequence[CFDatasetEnum]


class CFDataset(MetadataMixin, IndependentABC):
    @classmethod
    def from_zarr(cls, path: StrPath, dvars: VariableLike) -> CFDataset:
        return cls.from_dependant(xr.open_zarr(path), dvars)

    @classmethod
    def from_dependant(cls, ds: xr.Dataset, dvars: VariableLike) -> CFDataset:
        return cls(set_independent(ds), dvars)

    def __init__(self, ds: xr.Dataset, dvars: VariableLike) -> None:
        # assert is_independent(ds)

        super().__init__()
        self._enum, self._dvars = _, dvars = self._validate_variables(dvars)
        self._ds = ds = ds[dvars]

    @staticmethod
    def _validate_variables(dvars: VariableLike) -> tuple[type[CFDatasetEnum], list[CFDatasetEnum]]:
        if isinstance(dvars, type):
            assert issubclass(dvars, CFDatasetEnum)
            enum = dvars
            dvars = list(dvars)
        elif isinstance(dvars, CFDatasetEnum):
            enum = dvars.__class__
            dvars = [dvars]
        else:
            enum = dvars[0].__class__
            dvars = list(dvars)

        for dvar in dvars:
            assert isinstance(dvar, enum)
        return enum, dvars

    @property
    def ds(self) -> xr.Dataset:  # abc
        return self._ds

    @property
    def enum(self) -> type[CFDatasetEnum]:
        return self._enum

    @property
    def metadata(self):
        return self._enum.md

    @property
    def dvars(self) -> list[CFDatasetEnum]:
        return self._dvars


cf = CFDataset.from_zarr(urma_store, URMAEnum("ceiling", "vis"))

In [8]:
ceil = cf.dvars[0]
print(ceil)
ceil.crs

ceiling


<Projected CRS: {"$schema": "https://proj.org/schemas/v0.2/projjso ...>
Name: NDFD
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Conic Conformal (1SP)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
import matplotlib.pyplot as plt


class Dataset:
    """
    # CONUS and Northern Hemisphere Grids

    https://graphical.weather.gov/docs/ndfdSRS.htm#:~:text=The%20NDFD%20uses%20the%20World%20Geodetic%20System,1984%20%28WGS84%29%20ellipsoid%20for%20its%20horizontal%20datum.

    Grid Parameter	    CONUS 2.5km
    Number of Points	2953665
    Projection Type	    Lambert Conformal
    Shape of Earth      Sphere
    Earth Radius	    6371.2 km
    Number of Points on the parallel	2145
    Number of Points on the Meridian	1377
    Latitude1:	20.191999
    Longitude1:	238.445999
    u/v vectors relative to:	easterly/northerly
    Dx	2539.703 m
    Dy	2539.703 m
    GRIB2 grid, scan mode	64 (0100)
    Scan i/x direction	positive
    Scan j/y direction	positive
    Consecutive points in	i/x direction
    Adjacent rows scan in	same direction
    Mesh Latitude	25
    Orientation Longitude	265
    Which Pole is on the Plane	north
    Is Projection Bi-polar	no
    Tangent Latitude1	25
    Tangent Latitude2	25
    Southern Latitude	-90
    Southern Longitude	0
    """

    cf = types.MappingProxyType(
        {
            "geographic_crs_name": "NDFD CONUS 2.5km Lambert Conformal Conic",
            "projected_crs_name": "NDFD",
            "semi_major_axis": 6378137.0,
            "semi_minor_axis": 6356752.31424518,
            "inverse_flattening": 298.25722356301,
            "reference_ellipsoid_name": "WGS 84",
            "longitude_of_prime_meridian": 0.0,
            "prime_meridian_name": "Greenwich",
            "horizontal_datum_name": "WGS84",
            "latitude_of_projection_origin": 20.191999,
            "longitude_of_projection_origin": 238.445999,
            "false_easting": 0.0,
            "false_northing": 0.0,
        }
    )

    def __init__(self, ds: xr.Dataset):
        self.da = da = ds.to_array().transpose("x", "y", ...)
        self.lons = lons = (da["longitude"].to_numpy() + 180) % 360 - 180
        self.lats = lats = da["latitude"].to_numpy()

        self._source_definition = pyresample.geometry.AreaDefinition(
            self.cf["geographic_crs_name"],
            "National Digital Forecast Database Grid",
            self.cf["projected_crs_name"],
            self.get_crs("lambert_conformal_conic", standard_parallel=25),
            da["y"].size,
            da["x"].size,
            area_extent=self.area_extent,
            lons=lons,
            lats=lats,
        )

    def get_source_definition(self) -> pyresample.geometry.AreaDefinition:
        return self._source_definition

    def get_target_definition(
        self, latitude: float, longitude: float, width: float, height: float, area_extent: list[float]
    ) -> pyresample.geometry.AreaDefinition:
        crs = self.get_crs("lambert_azimuthal_equal_area", latitude=latitude, longitude=longitude)
        return pyresample.geometry.AreaDefinition(
            "target_projection",
            "description",
            None,
            crs,
            width=width,
            height=height,
            area_extent=area_extent,
        )

    def get_crs(
        self, grid_mapping_name: str, *, latitude: float | None = None, longitude: float | None = None, **kwargs
    ) -> pyproj.CRS:
        origin: dict[str, Any] = {"grid_mapping_name": grid_mapping_name, "units": "m"}
        if latitude is not None:
            origin["latitude_of_projection_origin"] = latitude

        if longitude is not None:
            origin["longitude_of_projection_origin"] = longitude

        return pyproj.CRS.from_cf(self.cf | origin | kwargs)

    @property
    def area_extent(self) -> list[float]:
        return [
            self.lons.min(),
            self.lats.min(),
            self.lons.max(),
            self.lats.max(),
        ]

    def resample_on_center(
        self,
        longitude: float,
        latitude: float,
        *,
        width=256,
        height=256,
        dx=100,
        dy=100,
        scale_x=1,
        scale_y=1,
        units="km",
    ):
        if units == "km":
            dx *= 1000
            dy *= 1000

        dx /= 2
        dy /= 2

        height *= scale_y
        width *= scale_x

        source = self.get_source_definition()
        data = self.da.to_numpy()

        area_extent = [-dx * scale_x, -dy * scale_y, dx * scale_x, dy * scale_y]
        target = self.get_target_definition(latitude, longitude, width, height, area_extent)

        return pyresample.kd_tree.resample_nearest(source, data, target, radius_of_influence=50000)


dataset = Dataset(get_urma().isel(time=0))
data = dataset.resample_on_center(longitude=-89.835, latitude=38.54)
H, W, C = data.shape

fig, axes = plt.subplots(1, C, figsize=(20, 5))

for i in range(C):
    ax = axes[i]
    ax.imshow(data[:, :, i], origin="upper", cmap="terrain")
    ax.set_xticks([])
    ax.set_yticks([])